In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 8.0)
import mayavi.mlab as mlab
import matplotlib.pyplot as plt
import numpy as np

import sparse_transforms as spt

# Initialization

## For visualization 

In [ ]:
def viz3D(grids, V, X_label='X', Y_label='Y', Z_label='Z', title='3D Visualization', interpolation='linear'):

    mlab.figure()

    X, Y, Z = grids.expanded
    src = mlab.pipeline.scalar_field(X, Y, Z, V)
    src.spacing = [1, 1, 1]
    src.update_image_data = True    
    ipw_x = mlab.pipeline.image_plane_widget(src, plane_orientation='x_axes')
    ipw_x.ipw.reslice_interpolate = interpolation
    ipw_x.ipw.texture_interpolate = False
    ipw_y = mlab.pipeline.image_plane_widget(src, plane_orientation='y_axes')
    ipw_y.ipw.reslice_interpolate = interpolation
    ipw_y.ipw.texture_interpolate = False
    ipw_z = mlab.pipeline.image_plane_widget(src, plane_orientation='z_axes')
    ipw_z.ipw.reslice_interpolate = interpolation
    ipw_z.ipw.texture_interpolate = False
    mlab.colorbar()
    mlab.outline()
    mlab.xlabel(X_label)
    mlab.ylabel(Y_label)
    mlab.zlabel(Z_label)

    limits = []
    for grid in (X, Y, Z):
        limits += [grid.min()]
        limits += [grid.max()]
    mlab.axes(ranges=limits)
    mlab.title(title)
    
    mlab.show()

## Create the base grids object

In [ ]:
Y = np.linspace(0, 10, 50)
X = np.linspace(0, 10, 50)
Z = np.linspace(0, 10, 50)
        
grids = spt.Grids(Y, X, Z)

## Create a base volume 

In [ ]:
Y, X, Z = grids.expanded
V = np.ones_like(Y)
V[Y>8] = 0
V[X>8] = 0
V[Z>8] = 0
V[Y<2] = 0
V[X<2] = 0
V[Z<2] = 0

In [ ]:
viz3D(grids, V)

# Transforms 

## Translation

In [ ]:
viz3D(grids.translate((5, 5, 5)), V)

## Rotation 

In [ ]:
Hrot = spt.rotationTransform(grids, rotation=(0, 0, np.pi/4))

In [ ]:
Vrot = Hrot * V

In [ ]:
viz3D(Hrot.out_grids, Vrot)

## Translation + Rotation

In [ ]:
#
# Create a grids centered at the cube
#
translated_grids = grids.translate((5, 5, 5))

Hrot_trans = spt.rotationTransform(
    translated_grids,
    rotation=(0, 0, np.pi/4),
    out_grids=translated_grids
)

In [ ]:
Vrot_trans = Hrot_trans * V

In [ ]:
#
# Visualize in the original grids
#
viz3D(grids, Vrot_trans)

## Integral 

In [ ]:
Hint = spt.integralTransform(grids, axis=0)
Hrot_int = spt.integralTransform(Hrot.out_grids, axis=1)

In [ ]:
Iint = Hint * V
Irot_int = Hrot_int * Vrot

In [ ]:
plt.imshow(Iint)
plt.colorbar()

In [ ]:
plt.imshow(Irot_int)
plt.colorbar()

## Cumsum Transform

In [ ]:
Hcs = spt.cumsumTransform(grids, axis=0)
Hrot_cs = spt.cumsumTransform(Hrot.out_grids, axis=0)

In [ ]:
Vcs = Hcs * V
Vrot_cs = Hrot_cs * Vrot

In [ ]:
viz3D(grids, Vcs)

In [ ]:
viz3D(Hrot.out_grids, Vrot_cs)